<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/Copy_of_Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install censusgeocode
!pip install datatable
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session
import datetime
from datetime import datetime,timedelta
!pip install pytz
import pytz
import datatable as dt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.3 MB/s 
     |████████████████████████████████| 62 kB 731 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96.9 MB 98 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Lookin

In [2]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()


True

In [3]:
import datetime
n_days_ago = datetime.date.today() - datetime.timedelta(days=180)
dt_string = n_days_ago.strftime("%Y-%m-%dT%H:%M:%S.000")

dt_string

'2022-01-15T00:00:00.000'

In [ ]:
ListingId_result = rets_client.search(resource = 'Property', resource_class='Property',dmql_query='(StatusChangeTimestamp = {}+),(MlsStatus = SLD)'.format(dt_string))
lis = []
di ={}
df = pd.DataFrame()
for item in ListingId_result:
  if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence' and item['StateOrProvince']== 'Texas' and item['BedroomsTotal'] == '4' or item['BedroomsTotal'] == '3'):
   
    df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) 
    di = item   
for key in di:
  lis.append(key)

df = df.T
df.columns = lis
df['new_col'] = df['SubdivisionName'].astype(str).str[0:10]


n = len(pd.unique(df['new_col']))
result = df[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','LotSizeAcres','new_col']]  
# n = df.groupby(by=['new_col'])["new_col"].count() 
n_by_state = df.groupby(by=['new_col','BedroomsTotal'])["ClosePrice"].sum().reset_index().groupby('new_col').mean()
print(n_by_state.head(10))
result.to_csv('SLD.csv')
#print(di)
